In [1]:
import numpy as np
import pandas as pd 
import scipy as sp 
import matplotlib.pyplot as plt 
import seaborn as sns 
from tqdm import tqdm

from utils import *
from plot_stats import *

from hybrid_sim import *
from hybrid_fit import * 
from hybrid_predict import *

from nwsls_sim import *

from logistic_regression import *
from rnn_helper import *
from tensorflow.python.keras.losses import binary_crossentropy

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
#config path to save data
data_path_root = f'data'

#config num of agent to simulate
num_of_agents = 50

#config num of blocks for each agent 
num_of_simulation = 2
# for cross valdation 
array = np.arange(num_of_simulation)
cv = [np.roll(array,i) for i in range(num_of_simulation)]
cv = np.array(cv)

#config num of trails for each agent
num_of_trials = 200

# create transtion probs
transtion_probs = np.array([[.8,.2],
                            [.2,.8]])

# config RNN hyperparameters
# lr = 
# n_cells = 

### Model Hybrid Simulation & Fit

In [ ]:
#list to store paramters
parameters = []
data_per_agent = []

data_results = {
               'agent': [], 
               'fit_parameters_hybrid': [], 
               'train_nlp_hybrid' : [], 
               'test_acc_hybrid': [],
               'test_nlp_hybrid': [], 
    
               'fit_parameters_logistic_regression': [], 
               'train_nlp_logistic_regression' : [], 
               'test_acc_logistic_regression': [],
               'test_nlp_logistic_regression': [],
    
               'train_acc_rnn': [], 
               'train_nlp_rnn' : [], 
               'test_acc_rnn': [],
               'test_nlp_rnn': []
        }



# sim hybrid agent and fit on hybrid logistic_regression and RNN
for agent in tqdm(range(num_of_agents)):
    data = []
    param = configuration_parameters_hybrid()
    parameters.append(param)
    
    print(f'***** Sim Agent = {agent} *****')
    # simulations
    for i in range(num_of_simulation):

        # create rewards probs 
        reward_probs = create_reward_probs(num_of_trials,50,0.2,0.8)
        df = hybrid_sim(
                        param,
                        num_of_trials,
                        transtion_probs,
                        reward_probs
        ) 
        data.append(df)
    data_per_agent.append(data)
    
    print(f'*** Fit ***')
    for n,t in enumerate(cv):
        data_results['agent'].append(agent)
        print(f'* loocv block {n} *')
        train_arr = t[0:-1]
        test_arr = t[-1:]
       
        # split train and test data
        train_data = [data_per_agent[agent][sim] for sim in train_arr]
        train_data = pd.concat(train_data) 
        train_data.reset_index(inplace=True)
        n_train = len(train_data)

        test_data = [data_per_agent[agent][sim] for sim in test_arr]
        test_data = pd.concat(test_data) 
        test_data.reset_index(inplace=True)
        n_test = len(test_data)

        print('* fit Hybrid *')
        res = hybrid_fit(train_data,3)
        data_results['fit_parameters_hybrid'].append(res.x)
        
        # Train negative log probability
        data_results['train_nlp_hybrid'].append(res.fun/n_train)
        
        # Test Accuracy and loss
        accuracy, p_0 = hybrid_predict(res.x,test_data)
        
        # Test Accuracy
        data_results['test_acc_hybrid'].append(accuracy/n_test)
        
        # Test negative log probability
        running_loss = 0
        for row in test_data.itertuples(index=True, name='Pandas'):
            running_loss += binary_crossentropy([row.action_stage_1],[1-p_0[row.Index]])
        data_results['test_nlp_hybrid'].append((running_loss/n_test).numpy())
        
      
        print('* fit logistic_regression *')
        X , y = preprocess_logistic_regression(train_data)
        model, intercept, coef = fit_logistic_regression(X,y)
        data_results['fit_parameters_logistic_regression'].append([intercept,coef])

        # Train negative log probability
        nlp = nlp_logistic_regression(model,X,y)
        data_results['train_nlp_logistic_regression'].append((nlp/n_train).numpy())
        
        # test data 
        X, y = preprocess_logistic_regression(test_data)
        # Test Accuracy
        accuracy = model.score(X,y)
        data_results['test_acc_logistic_regression'].append(accuracy)
        
        # Test negative log probability
        nlp = nlp_logistic_regression(model,X,y)
        data_results['test_nlp_logistic_regression'].append((nlp/n_test).numpy())

        print('* fit RNN *')   
        model, sim_model, hist = train_a_model_gru(
                                              train_data,
                                              train_arr,
                                              test_data,
                                              test_arr,
                                              num_of_trials=num_of_trials,
                                              n_cells=5,
                                              lr=0.001,
                                              epochs=400,
                                              batch_size=100,
                                              cp_callback=True
        )

        data_results['train_acc_rnn'].append(hist.history['accuracy'][-1])
        data_results['train_nlp_rnn'].append(hist.history['binary_crossentropy'][-1])
        data_results['test_acc_rnn'].append(hist.history['val_accuracy'][-1])
        data_results['test_nlp_rnn'].append(hist.history['val_binary_crossentropy'][-1])

        print('* on policy RNN *')
        df, _ = on_policiy_tst(
                        sim_model,
                        num_of_trials=400,
                        reversal_each_trials=100,
                        n_cells=5
        )
        df.to_csv(f'{data_path_root}/hybrid/results/rnn_on_policiy_{agent}_cv_{i}.csv')

# save data 
df = pd.DataFrame(parameters)
df.to_csv(f'{data_path_root}/hybrid/sim_data/hybrid_paramters.csv')
for a in range(num_of_agents):
    for s in range(num_of_simulation):
        data_per_agent[a][s].to_csv(f'{data_path_root}/hybrid/sim_data/hybrid_agent_{a}_sim_{s}.csv')
        
df = pd.DataFrame(data_results)
df.to_csv(f'{data_path_root}/hybrid/results/hybrid_fit_dataframe.csv')

X = []
for a in range(num_of_agents):
    for s in range(num_of_simulation):
        update_data_frame(data_per_agent[a][s])
        X.append(data_per_agent[a][s])
    
X = pd.concat(X) 
print('Hybrid')
print(f'interaction = {calc_interaction_effect(X)}')
print(f'main = {calc_main_effect(X)}')
re = X['reward'].mean()
print(f'mean reward = {re}')
plot_stay_probs(X)        

  0%|                                                                                           | 0/50 [00:00<?, ?it/s]

***** Sim Agent = 0 *****
*** Fit ***
* loocv block 0 *
* fit Hybrid *
* fit logistic_regression *
* fit RNN *
* on policy RNN *
* loocv block 1 *
* fit Hybrid *
* fit logistic_regression *
* fit RNN *
* on policy RNN *


  2%|█▋                                                                                 | 1/50 [00:23<18:52, 23.12s/it]

***** Sim Agent = 1 *****
*** Fit ***
* loocv block 0 *
* fit Hybrid *
* fit logistic_regression *
* fit RNN *
* on policy RNN *
* loocv block 1 *
* fit Hybrid *
* fit logistic_regression *
* fit RNN *
* on policy RNN *


  4%|███▎                                                                               | 2/50 [00:44<17:33, 21.94s/it]

***** Sim Agent = 2 *****
*** Fit ***
* loocv block 0 *
* fit Hybrid *
* fit logistic_regression *
* fit RNN *


### Model nwsls simulation & fit

In [ ]:
#list to store paramters
parameters = []
data_per_agent = []

data_results = {
               'agent': [], 
               'fit_parameters_hybrid': [], 
               'train_nlp_hybrid' : [], 
               'test_acc_hybrid': [],
               'test_nlp_hybrid': [], 
    
               'fit_parameters_logistic_regression': [], 
               'train_nlp_logistic_regression' : [], 
               'test_acc_logistic_regression': [],
               'test_nlp_logistic_regression': [],
    
               'train_acc_rnn': [], 
               'train_nlp_rnn' : [], 
               'test_acc_rnn': [],
               'test_nlp_rnn': []
        }


# sim nwsls agent and fit on hybrid logistic_regression and RNN models
for agent in tqdm(range(num_of_agents)):
    data = []
    param = configuration_parameters_nwsls()
    parameters.append(param)
    
    print(f'***** Sim Agent = {agent} *****')
    # simulations
    for i in range(num_of_simulation):

        # create rewards probs 
        reward_probs = create_reward_probs(num_of_trials,50,0.2,0.8)
        df = nwsls_sim(
                    param,
                    num_of_trials,
                    transtion_probs,
                    reward_probs
        ) 
        data.append(df)
    data_per_agent.append(data)
    
    print(f'*** Fit ***')
    # fit
    for n,t in enumerate(cv):
        data_results['agent'].append(agent)
        print(f'* loocv block {n} *')
        train_arr = t[0:-1]
        test_arr = t[-1:]
       
        # split train and test data
        train_data = [data_per_agent[agent][sim] for sim in train_arr]
        train_data = pd.concat(train_data) 
        train_data.reset_index(inplace=True)
        n_train = len(train_data)

        test_data = [data_per_agent[agent][sim] for sim in test_arr]
        test_data = pd.concat(test_data) 
        test_data.reset_index(inplace=True)
        n_test = len(test_data)

        print('* fit Hybrid *')
        res = hybrid_fit(train_data,3)
        data_results['fit_parameters_hybrid'].append(res.x)
        
        # Train negative log probability
        data_results['train_nlp_hybrid'].append(res.fun/n_train)
        
        # Test Accuracy and loss
        accuracy, p_0 = hybrid_predict(res.x,test_data)
        
        # Test Accuracy
        data_results['test_acc_hybrid'].append(accuracy/n_test)
        
        # Test negative log probability
        running_loss = 0
        for row in test_data.itertuples(index=True, name='Pandas'):
            running_loss += binary_crossentropy([row.action_stage_1],[1-p_0[row.Index]])
        data_results['test_nlp_hybrid'].append((running_loss/n_test).numpy())
        
      
        print('* fit logistic_regression *')
        X , y = preprocess_logistic_regression(train_data)
        model, intercept, coef = fit_logistic_regression(X,y)
        data_results['fit_parameters_logistic_regression'].append([intercept,coef])

        # Train negative log probability
        nlp = nlp_logistic_regression(model,X,y)
        data_results['train_nlp_logistic_regression'].append((nlp/n_train).numpy())
        
        # test data 
        X, y = preprocess_logistic_regression(test_data)
        # Test Accuracy
        accuracy = model.score(X,y)
        data_results['test_acc_logistic_regression'].append(accuracy)
        
        # Test negative log probability
        nlp = nlp_logistic_regression(model,X,y)
        data_results['test_nlp_logistic_regression'].append((nlp/n_test).numpy())

        print('* fit RNN *')   
        model, sim_model, hist = train_a_model_gru(
                                              train_data,
                                              train_arr,
                                              test_data,
                                              test_arr,
                                              num_of_trials=num_of_trials,
                                              n_cells=5,
                                              lr=0.001,
                                              epochs=400,
                                              batch_size=100,
                                              cp_callback=True
        )

        data_results['train_acc_rnn'].append(hist.history['accuracy'][-1])
        data_results['train_nlp_rnn'].append(hist.history['binary_crossentropy'][-1])
        data_results['test_acc_rnn'].append(hist.history['val_accuracy'][-1])
        data_results['test_nlp_rnn'].append(hist.history['val_binary_crossentropy'][-1])

        print('* on policy RNN *')
        df, _ = on_policiy_tst(
                        sim_model,
                        num_of_trials=400,
                        reversal_each_trials=100,
                        n_cells=5
        )
        df.to_csv(f'{data_path_root}/nwsls/results/rnn_on_policiy_{agent}_cv_{i}.csv')

# save data 
df = pd.DataFrame(parameters)
df.to_csv(f'{data_path_root}/nwsls/sim_data/nwsls_paramters.csv')
for a in range(num_of_agents):
    for s in range(num_of_simulation):
        data_per_agent[a][s].to_csv(f'{data_path_root}/nwsls/sim_data/nwsls_agent_{a}_sim_{s}.csv')
        
df = pd.DataFrame(data_results)
df.to_csv(f'{data_path_root}/nwsls/results/nwsls_fit_dataframe.csv')

X = []
for a in range(num_of_agents):
    for s in range(num_of_simulation):
        update_data_frame(data_per_agent[a][s])
        X.append(data_per_agent[a][s])
    
X = pd.concat(X) 
print('nwsls')
print(f'interaction = {calc_interaction_effect(X)}')
print(f'main = {calc_main_effect(X)}')
re = X['reward'].mean()
print(f'mean reward = {re}')
plot_stay_probs(X)        